In [21]:
import numpy as np 
import pandas as pd
import re
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from scipy import special
from tqdm import tqdm
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.impute import SimpleImputer 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score, classification_report, accuracy_score, roc_curve
import matplotlib.pyplot as plt
import optuna
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMClassifier, LGBMRegressor
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.feature_selection import RFE

warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:20,.2f}'.format)

In [22]:
X_train = pd.read_csv('/Users/sakshamjain/Desktop/Projects/JAIN-WIN/X_train_corr.csv')
y_train = pd.read_csv('/Users/sakshamjain/Desktop/Projects/JAIN-WIN/y_train_large.csv').squeeze()
X_test = pd.read_csv('/Users/sakshamjain/Desktop/Projects/JAIN-WIN/X_test_corr.csv')
y_test = pd.read_csv('/Users/sakshamjain/Desktop/Projects/JAIN-WIN/y_test_large.csv').squeeze()

In [23]:
X_train.head()

,age,bmi,elective_surgery,gender,height,icu_id,pre_icu_los_days,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,icu_id_counts,age_counts,nan_counts,sq_age,sq_bmi,bmi_age,weight_age,comorbidity_score,diabete_risk,gcs_sum,apache_2_diagnosis_type,bmi_type,height_type,weight_type,age_type,gcs_sum_type,apache_3j_diagnosis_split1,apache_2_diagnosis_split1,profile,diff_bmi,bilirubin_d1_h1_max_eq,bilirubin_d1_h1_min_eq,bilirubin_h1_value_range,bilirubin_h1_zero_range,bilirubin_h1_mean,bilirubin_h1_std,bilirubin_h1_max_apache_3j_diagnosis_mean,bilirubin_h1_max_apache_3j_diagnosis_diff,bilirubin_h1_max_apache_3j_diagnosis_std,bilirubin_h1_max_apache_3j_diagnosis_norm_std,bilirubin_h1_max_apache_3j_diagnosis_rank,bilirubin_h1_max_apache_3j_diagnosis_count,bilirubin_h1_max_apache_3j_diagnosis_norm_rank,bilirubin_h1_max_apache_3j_diagnosis_skew,bilirubin_h1_min_apache_3j_diagnosis_mean,bilirubin_h1_min_apache_3j_diagnosis_diff,bilirubin_h1_min_apache_3j_diagnosis_std,bilirubin_h1_min_apache_3j_diagnosis_norm_std,bilirubin_h1_min_apache_3j_diagnosis_rank,bilirubin_h1_min_apache_3j_diagnosis_norm_rank,bilirubin_h1_min_apache_3j_diagnosis_skew,bilirubin_h1_max_profile_mean,bilirubin_h1_max_profile_diff,bilirubin_h1_max_profile_std,bilirubin_h1_max_profile_norm_std,bilirubin_h1_max_profile_rank,bilirubin_h1_max_profile_count,bilirubin_h1_max_profile_norm_ran

In [7]:
print("Training LightGBM model...")
model_all = lgb.LGBMClassifier( random_state=69, n_jobs=-1, force_col_wise=True )
model_all.fit(X_train, y_train)
print("Model training completed.")

# Predictions and Probabilities
y_pred_all = model_all.predict(X_test)
y_proba_all = model_all.predict_proba(X_test)[:, 1]

# Metrics
print("Classification Report for All Features:")
print(classification_report(y_test, y_pred_all))
auc_all = roc_auc_score(y_test, y_proba_all)
print(f"AUC-ROC for All Features: {auc_all:.8f}\n")

Training LightGBM model...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 19677, number of negative: 71432
[LightGBM] [Info] Total Bins 850113
[LightGBM] [Info] Number of data points in the train set: 91109, number of used features: 3698
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215972 -> initscore=-1.289296
[LightGBM] [Info] Start training from score -1.289296
Model training completed.
Classification Report for All Features:
              precision    recall  f1-score   support

         0.0       0.87      0.94      0.90     30574
         1.0       0.69      0.50      0.58      8474

    accuracy                           0.84     39048
   macro avg       0.78      0.72      0.74     39048
weighted avg       0.83      0.84      0.83     39048

AUC-ROC for All Features: 0.87206368



In [8]:
importances = model_all.feature_importances_
feature_importances = pd.DataFrame({
    'feature': X_train.columns,
    'importance': importances
})

feature_importances.sort_values(by='importance', ascending=False, inplace=True)
feature_importances['cumulative_importance'] = feature_importances['importance'].cumsum() / feature_importances['importance'].sum()
selected_features = feature_importances[feature_importances['cumulative_importance'] <= 0.95]['feature'].tolist()
num_features_selected = len(selected_features)
print(f"Selected {num_features_selected} features based on cumulative importance (>=95%).\n")

Selected 1277 features based on cumulative importance (>=95%).



In [9]:
X_train_cum95 = X_train[selected_features]
X_test_cum95 = X_test[selected_features]

In [30]:
def find_dataframes():
    # This will check for variables that are instances of pd.DataFrame in the global scope
    return {name: obj for name, obj in globals().items() if isinstance(obj, pd.DataFrame)}

def print_memory_usage_of_dataframes():
    dataframes = find_dataframes()
    total_memory = 0
    print("Memory usage of dataframes (in GB):")
    for name, df in dataframes.items():
        mem_usage = df.memory_usage(deep=True).sum() / 1024 ** 3  # Convert bytes to gigabytes
        total_memory += mem_usage
        print(f"{name}: {mem_usage:.6f} GB")
    print(f"Total memory used by dataframes: {total_memory:.6f} GB")

print_memory_usage_of_dataframes()

Memory usage of dataframes (in GB):
_: 0.000137 GB
_3: 0.000137 GB
feature_importances: 0.000371 GB
X_train: 2.499566 GB
X_test: 1.071278 GB
_23: 0.000137 GB
Total memory used by dataframes: 3.571627 GB


In [29]:
del X_test_rfe

In [14]:
print("Training LightGBM model on cumulative 95% important features...")
model_cum95 = lgb.LGBMClassifier( random_state=69, n_jobs=-1, force_col_wise=True )
model_cum95.fit(X_train_cum95, y_train)
print("Model trained on cumulative 95% important features.\n")

# Predictions and Probabilities
y_pred_cum95 = model_cum95.predict(X_test_cum95)
y_proba_cum95 = model_cum95.predict_proba(X_test_cum95)[:, 1]

# Metrics
print("Classification Report for Cumulative 95% Important Features:")
print(classification_report(y_test, y_pred_cum95))
auc_cum95 = roc_auc_score(y_test, y_proba_cum95)
print(f"AUC-ROC for Cumulative 95% Important Features: {auc_cum95:.8f}\n")

Training LightGBM model on cumulative 95% important features...
[LightGBM] [Info] Number of positive: 19677, number of negative: 71432
[LightGBM] [Info] Total Bins 311277
[LightGBM] [Info] Number of data points in the train set: 91109, number of used features: 1277
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215972 -> initscore=-1.289296
[LightGBM] [Info] Start training from score -1.289296
Model trained on cumulative 95% important features.

Classification Report for Cumulative 95% Important Features:
              precision    recall  f1-score   support

         0.0       0.87      0.94      0.90     30574
         1.0       0.69      0.50      0.58      8474

    accuracy                           0.84     39048
   macro avg       0.78      0.72      0.74     39048
weighted avg       0.83      0.84      0.83     39048

AUC-ROC for Cumulative 95% Important Features: 0.87235650



In [16]:
rfe = RFE(estimator=model_cum95, n_features_to_select=639, step=0.05)
print("Fitting RFE...")
rfe.fit(X_train_cum95, y_train)
print("RFE fitting completed.\n")

# Get the selected features
selected_features_rfe = X_train_cum95.columns[rfe.support_].tolist()
print(f"Selected {len(selected_features_rfe)} features after RFE: {selected_features_rfe}\n")

X_train_rfe = X_train_cum95[selected_features_rfe]
X_test_rfe = X_test_cum95[selected_features_rfe]

# Train LightGBM on RFE-selected Features
print("Training LightGBM model on RFE-selected features...")
model_rfe = lgb.LGBMClassifier( random_state=69, n_jobs=-1, force_col_wise=True )
model_rfe.fit(X_train_rfe, y_train)
print("Model trained on RFE-selected features.\n")

# Predictions and Probabilities
y_pred_rfe = model_rfe.predict(X_test_rfe)
y_proba_rfe = model_rfe.predict_proba(X_test_rfe)[:, 1]

# Metrics
print("Classification Report for RFE-selected Features:")
print(classification_report(y_test, y_pred_rfe))
auc_rfe = roc_auc_score(y_test, y_proba_rfe)
print(f"AUC-ROC for RFE-selected Features: {auc_rfe:.8f}\n")

Fitting RFE...
[LightGBM] [Info] Number of positive: 19677, number of negative: 71432
[LightGBM] [Info] Total Bins 311277
[LightGBM] [Info] Number of data points in the train set: 91109, number of used features: 1277
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215972 -> initscore=-1.289296
[LightGBM] [Info] Start training from score -1.289296
[LightGBM] [Info] Number of positive: 19677, number of negative: 71432
[LightGBM] [Info] Total Bins 295793
[LightGBM] [Info] Number of data points in the train set: 91109, number of used features: 1214
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215972 -> initscore=-1.289296
[LightGBM] [Info] Start training from score -1.289296
[LightGBM] [Info] Number of positive: 19677, number of negative: 71432
[LightGBM] [Info] Total Bins 280191
[LightGBM] [Info] Number of data points in the train set: 91109, number of used features: 1151
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215972 -> initscore=-1.289296
[LightGBM] [Info] Start trainin

In [32]:
model_hehe = lgb.LGBMClassifier( random_state=69, n_jobs=-1, force_col_wise=True )

rfe = RFE(estimator=model_hehe, n_features_to_select=1277, step=0.05)
print("Fitting RFE...")
rfe.fit(X_train, y_train)
print("RFE fitting completed.\n")

# Get the selected features
selected_features_rfe = X_train.columns[rfe.support_].tolist()
print(f"Selected {len(selected_features_rfe)} features after RFE: {selected_features_rfe}\n")

X_train_rfe = X_train[selected_features_rfe]
X_test_rfe = X_test[selected_features_rfe]

# Train LightGBM on RFE-selected Features
print("Training LightGBM model on RFE-selected features...")
model_rfe = lgb.LGBMClassifier( random_state=69, n_jobs=-1, force_col_wise=True )
model_rfe.fit(X_train_rfe, y_train)
print("Model trained on RFE-selected features.\n")

# Predictions and Probabilities
y_pred_rfe = model_rfe.predict(X_test_rfe)
y_proba_rfe = model_rfe.predict_proba(X_test_rfe)[:, 1]

# Metrics
print("Classification Report for RFE-selected Features:")
print(classification_report(y_test, y_pred_rfe))
auc_rfe = roc_auc_score(y_test, y_proba_rfe)
print(f"AUC-ROC for RFE-selected Features: {auc_rfe:.8f}\n")

Fitting RFE...
[LightGBM] [Info] Number of positive: 19677, number of negative: 71432
[LightGBM] [Info] Total Bins 850113
[LightGBM] [Info] Number of data points in the train set: 91109, number of used features: 3698
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215972 -> initscore=-1.289296
[LightGBM] [Info] Start training from score -1.289296
[LightGBM] [Info] Number of positive: 19677, number of negative: 71432
[LightGBM] [Info] Total Bins 806550
[LightGBM] [Info] Number of data points in the train set: 91109, number of used features: 3514
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215972 -> initscore=-1.289296
[LightGBM] [Info] Start training from score -1.289296
[LightGBM] [Info] Number of positive: 19677, number of negative: 71432
[LightGBM] [Info] Total Bins 763090
[LightGBM] [Info] Number of data points in the train set: 91109, number of used features: 3330
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215972 -> initscore=-1.289296
[LightGBM] [Info] Start trainin

: 